<a href="https://colab.research.google.com/github/TSOlami/lstm-trading-bot/blob/main/trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Modules

In [ ]:
%tensorflow_version 2.x

: 

#Install Dependencies

In [ ]:
%pip install pandas_ta keras

: 

#Imports

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from keras import optimizers
from keras.models import Model
from keras.layers import LSTM, Input, Dense, Dropout, Activation


: 

#Get Data

###Define data download params and download the data

In [ ]:
tickers = 'AAPL'
start = '2004-01-01'
end = '2024-01-01'

data = yf.download(tickers, start, end)
data.head()

: 

#Define indicators, target variables and clean the data

In [ ]:
# Calculate technical indicators
data['RSI'] = ta.rsi(data['Close'], length=14)
data['EMAF'] = ta.ema(data['Close'], length=9)
data['EMAM'] = ta.ema(data['Close'], length=21)
data['EMAS'] = ta.ema(data['Close'], length=50)
data['BollingerB'] = ta.bbands(data['Close'], length=20)['BBL_20_2.0']
data['Stoch'] = ta.stoch(data['High'], data['Low'], data['Close'])['STOCHk_14_3_3']
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
data['OBV'] = ta.obv(data['Close'], data['Volume'])
data['VWAP'] = ta.vwap(data['High'], data['Low'], data['Close'], data['Volume'])

# Prepare target variables for classification
data['Target'] = data['Adj Close'] - data.Open
data['Target'] = data['Target'].shift(-1)
data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]
data['TargetNextClose'] = data['Adj Close'].shift(-1)

# Drop NaN values and unnecessary columns
data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

# Prepare dataset for training
data_set = data.iloc[:, :-1]  # Exclude TargetNextClose column
pd.set_option('display.max_columns', None)

data_set.head(20)

: 

#Prepare data for training

In [ ]:
# Determine which columns to include in X
X_columns = ['Open', 'High', 'Low', 'Adj Close', 'RSI', 'EMAF', 'EMAM', 'EMAS', 'BollingerB', 'Stoch', 'ATR', 'OBV', 'VWAP']

scaler = MinMaxScaler(feature_range=(0,1))
data_set_scaled = scaler.fit_transform(data_set[X_columns])

backcandles = 30

X = np.array([data_set_scaled[i - backcandles:i, :].copy() for i in range(backcandles,len(data_set_scaled))])
print(X.shape)
yi = np.array(data_set_scaled[backcandles:,-1])
y = np.reshape(yi,(len(yi),1))


: 

#Split the data into train and test sets

In [ ]:
# Define the size of the training set
train_size = int(0.9 * len(X))

# Split the data into training and testing sets
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

: 

#The Model

In [ ]:
# Define the model architecture
input_shape = (backcandles, 13)  # Shape of input data
inputs = Input(shape=input_shape, name='input_layer')

# Add LSTM layers
x = LSTM(150, name='first_layer')(inputs)

# Add Dense layer
x = Dense(1, name='dense_layer')(x)

# Add Activation layer
output = Activation('linear', name='output')(x)

# Create the model
model = Model(inputs=inputs, outputs=output)

# Compile the model
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')

model.fit(x=X_train, y=y_train, batch_size=10, epochs=50, shuffle=False, validation_split = 0.1)

: 


#Training Results and Test


In [ ]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate accuracy (assuming this is a regression problem)
accuracy = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error (MAE):", accuracy)

# Print out some example predictions and actual values
print("Example predictions and actual values:")
for i in range(10):
    print("Prediction:", predictions[i][0], " - Actual:", y_test[i][0])


: 

#Graph

In [ ]:

# Plot some example predictions vs actual values
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'yellow', label = 'Actual Value')
plt.plot(predictions, color = 'green', label = 'Prediction')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Example Predictions vs Actual Values')
plt.legend()
plt.show()

: 

#Prediction and Accuracy Measurements

In [ ]:
# Make binary predictions based on the model's output
binary_predictions = np.where(predictions > 0.5, 1, 0)
binary_y_test = np.where(y_test > 0, 1, 0)

# Compute accuracy
accuracy = np.mean(binary_predictions == binary_y_test) * 100
print("Accuracy: {:.2f}%".format(accuracy))


: 